In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [ ]:
from src.loader import SlackDataLoader
from src.config import SLACK_DATA_PATH

# Initialize DataLoader
data_loader = SlackDataLoader(SLACK_DATA_PATH)

# Example usage
print("Channels:", data_loader.channels)
print("Users:", data_loader.users)

# Get messages from a specific channel
channel_name = "your_channel_name"
channel_messages = data_loader.get_channel_messages(channel_name)
print(f"Messages from {channel_name}:", channel_messages)

# User map
print("User Names by ID:", data_loader.user_names_by_id)
print("User IDs by Name:", data_loader.user_ids_by_name)


### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [ ]:
import json
import glob
import pandas as pd

def extract_info(row):
    return {
        'msg_type': row.get('type', ''),
        'msg_content': row.get('text', ''),
        'sender_name': row.get('user_profile', {}).get('real_name', 'Not provided'),
        'msg_sent_time': row.get('ts', ''),
        'msg_dist_type': row.get('blocks', [{}])[0].get('elements', [{}])[0].get('elements', [{}])[0].get('type', 'reshared'),
        'time_thread_start': row.get('thread_ts', 0),
        'reply_count': row.get('reply_count', 0),
        'reply_users_count': row.get('reply_users_count', 0),
        'reply_users': ','.join(row.get('reply_users', [])),
        'tm_thread_end': row.get('latest_reply', 0)
    }

def slack_parser(path_channel):
    dflist = []

    # Loop through each JSON file in the specified path
    for json_file in glob.glob(f"{path_channel}/*.json"):
        with open(json_file, 'r', encoding="utf8") as slack_data:
            try:
                data = json.load(slack_data)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in {json_file}: {e}")
                continue

        # Extract required information from each message
        messages = [extract_info(row) for row in data if 'bot_id' not in row]

        # Create a DataFrame from the extracted messages
        df = pd.DataFrame(messages)

        # Filter out rows where sender name is 'Not provided'
        df = df[df['sender_name'] != 'Not provided']

        # Append the DataFrame to the list
        dflist.append(df)

    # Concatenate all DataFrames in the list
    dfall = pd.concat(dflist, ignore_index=True)

    # Add a 'channel' column with the channel name
    dfall['channel'] = path_channel.split('/')[-1].split('.')[0]

    # Reset the index
    dfall = dfall.reset_index(drop=True)

    return dfall

# Specify the path to the directory containing JSON files
SLACK_DATA_PATH = "/home/habte/t"

# Call the slack_parser function
df_combined = slack_parser(SLACK_DATA_PATH)

# Print the combined DataFrame
print(df_combined)


In [ ]:
def convert_to_timestamp(column, data):
    """Convert from Unix time to readable timestamp."""
    if column in data.columns.values:
        timestamp_ = [datetime.datetime.fromtimestamp(float(time_unix)).strftime('%Y-%m-%d %H:%M:%S') if time_unix != 0 else 0 for time_unix in data[column]]
        return timestamp_
    else:
        print(f"{column} not in data")

def get_tagged_users(df):
    """Get all @ mentions in the messages."""
    return df['msg_content'].map(lambda x: re.findall(r'@U\w+', x))

def map_userid_to_realname(user_profile: dict, comm_dict: dict, plot=False):
    """
    Map Slack ID to real names.
    
    Args:
    user_profile: A dictionary that contains users' info such as real names.
    comm_dict: A dictionary that contains Slack ID and total messages sent by that Slack ID.
    plot: If True, plot the results.
    
    Returns:
    DataFrame with mapped user IDs to real names and message counts.
    """
    user_dict = {i: real_name for i, real_name in zip(user_profile['id'], [dict(user_profile['profile'])[i]['real_name'] for i in range(len(user_profile['profile']))])}
    
    ac_comm_dict = {user_dict[i]: comm_dict[i] for i in comm_dict if i in user_dict}
    ac_comm_dict = pd.DataFrame(data=zip(ac_comm_dict.keys(), ac_comm_dict.values()),
                                columns=['LearnerName', '# of Msg sent in Threads']).sort_values(by='# of Msg sent in Threads', ascending=False)
    
    if plot:
        ac_comm_dict.plot.bar(figsize=(15, 7.5), x='LearnerName', y='# of Msg sent in Threads')
        plt.title('Student based on Message sent in thread', size=20)
        
    return ac_comm_dict

def parse_slack_reaction(path, channel):
    """Parse Slack reactions from JSON files."""
    df_reaction = pd.DataFrame(columns=['reaction_name', 'reaction_count', 'reaction_users_count', 'message', 'user_id', 'channel'])

    for json_file in glob.glob(f"{path}*.json"):
        with open(json_file, 'r') as slack_data:
            data = json.load(slack_data)

            for message in data:
                if 'reactions' in message:
                    for reaction in message['reactions']:
                        df_reaction = df_reaction.append({
                            'reaction_name': reaction['name'],
                            'reaction_count': reaction['count'],
                            'reaction_users_count': len(reaction['users']),
                            'message': message['text'],
                            'user_id': message['user'],
                            'channel': channel
                        }, ignore_index=True) # type: ignore

    return df_reaction

def get_community_participation(path):
    """Get community participation from JSON files."""
    comm_dict = {}

    for json_file in glob.glob(f"{path}*.json"):
        with open(json_file, 'r') as slack_data:
            data = json.load(slack_data)

            for message in data:
                if 'replies' in message:
                    for reply in message['replies']:
                        comm_dict[reply['user']] = comm_dict.get(reply['user'], 0) + 1

    return comm_dict

# Usage:
directory_path = "/home/habte/t"
channel_name = "Random"

df_reaction = parse_slack_reaction(directory_path, channel_name)
comm_dict = get_community_participation(directory_path)

# Specify the path to the directory containing JSON files
path_to_json = "/home/habte/t"

channel_name = "t"

# Parse reactions from Slack data
df_reaction = parse_slack_reaction(path_to_json, channel_name)

comm_dict = get_community_participation(path_to_json)

# Display the dataframes
print("DataFrame df_reaction:")
print(df_reaction.head())

print("\nCommunity Participation Dictionary:")
print(comm_dict)



In [ ]:
df = combined_data 

def get_top_users_plot(data, channel='t', top_n=20):
    """Plot the top message senders in a channel."""
    top_users = data['sender_name'].value_counts()[:top_n]

    top_users.plot.bar(figsize=(15, 7.5))
    plt.title(f'Top {top_n} Message Senders in #{channel} channel', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18)
    plt.ylabel("Frequency", size=14)
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.show()


get_top_users_plot(df, channel='t', top_n=20)

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [ ]:
# which user has the highest number of reply counts?
# Assuming DataFrame is named 'df'
df = combined_data 
user_with_highest_replies = df[df['reply_count'] == df['reply_count'].max()]['sender_name'].iloc[0]
print(f"The user with the highest number of reply counts is: {user_with_highest_replies}")


In [ ]:
# Visualize reply counts per user per channel
import seaborn as sns
import matplotlib.pyplot as plt
df = combined_data 

# Assuming your DataFrame is named 'df'
plt.figure(figsize=(15, 7.5))
sns.barplot(x='sender_name', y='reply_count', hue='channel', data=df, ci=None)
plt.title('Reply Counts per User per Channel', size=15, fontweight='bold')
plt.xlabel('Sender Name', size=12); plt.ylabel('Reply Count', size=12)
plt.xticks(rotation=45, ha='right', size=10)
plt.legend(title='Channel', title_fontsize='12', loc='upper right')
plt.show()

In [ ]:
# which user has the highest number of reply counts?# what is the time range of the day that most messages are sent?

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
df['msg_sent_time'] = pd.to_datetime(df['msg_sent_time'])
df['hour_of_day'] = df['msg_sent_time'].dt.hour

# Plotting the distribution of messages across hours
plt.figure(figsize=(15, 7.5))
sns.countplot(x='hour_of_day', data=df)
plt.title('Distribution of Messages Across Hours of the Day', size=15, fontweight='bold')
plt.xlabel('Hour of Day', size=12)
plt.ylabel('Message Count', size=12)
plt.show()


In [11]:
# what kind of messages are replied faster than others?
import pandas as pd
df = combined_data 

def analyze_response_time(df):
    # Convert 'tm_thread_end' and 'msg_sent_time' to datetime format if they are in string format
    df['tm_thread_end'] = pd.to_datetime(df['tm_thread_end'], errors='coerce')
    df['msg_sent_time'] = pd.to_datetime(df['msg_sent_time'], errors='coerce')

    # Drop rows with missing values in 'tm_thread_end' or 'msg_sent_time'
    df = df.dropna(subset=['tm_thread_end', 'msg_sent_time'])

    # Calculate response time
    df['response_time'] = df['tm_thread_end'] - df['msg_sent_time']

    # Categorize messages based on keywords in the content
    df['message_category'] = df['msg_content'].apply(lambda x: 'urgent' if 'urgent' in x.lower() else 'normal')

    # Aggregate data based on message categories
    result = df.groupby('message_category')['response_time'].mean()

    # Print or return the result
    print(result)
    return result


analyze_response_time(df)

In [15]:
# Relationship between # of messages and # of reactions

In [16]:
# Classify messages into different categories such as questions, answers, comments, etc.
import re

def classify_message(message):
    # Define patterns for different message categories
    question_pattern = re.compile(r'\b(?:how|what|when|where|why)\b', flags=re.IGNORECASE)
    answer_pattern = re.compile(r'\b(?:here is|solution)\b', flags=re.IGNORECASE)
    comment_pattern = re.compile(r'\b(?:nice|great|explanation)\b', flags=re.IGNORECASE)

    # Check for patterns and assign categories
    if re.search(question_pattern, message):
        return 'Question'
    elif re.search(answer_pattern, message):
        return 'Answer'
    elif re.search(comment_pattern, message):
        return 'Comment'
    else:
        return 'Other'

# Example usage
df['message_category'] = df['message_content'].apply(classify_message)
print(df)


In [17]:
# Which users got the most reactions?

In [33]:
# Model topics mentioned in the channel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df['msg_content'] contains the messages
documents = df['msg_content'].dropna().values.astype('U')

# Create a document-term matrix using CountVectorizer
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
dtm = vectorizer.fit_transform(documents)

# Apply Latent Dirichlet Allocation
lda = LatentDirichletAllocation(n_components=5, random_state=42)
topics = lda.fit_transform(dtm)

# Display the top words for each topic
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    top_words_idx = topic.argsort()[:-10-1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"Topic #{topic_idx + 1}: {', '.join(top_words)}")

# Assign the dominant topic to each document
df['dominant_topic'] = topics.argmax(axis=1)

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?